In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score


In [3]:
file_path = r"../data/processed/sentiment_scoring.csv"
df = pd.read_csv(file_path)
df

,username,content,likecount,date,amend_weight_sum,length
0,SupremeWow,小米su7强势破圈太火爆了，雷总得抓紧提高产能啊,3025,2024/3/30 12:22,2.0,24
1,及米侠,感觉这次黑稿不太多，看来车圈还是更干净一些！,1725,2024/3/30 12:23,1.0,22
2,小麦Elite,小米手机1代回忆～,1020,2024/3/30 12:22,NaN,9
3,米12U,雷总辛苦了 感谢您为大家做了一辆Dream car,660,2024/3/30 12:22,1.0,25
4,Spider_boy666,雷总加电,650,2024/3/30 12:23,NaN,4
...,...,...,...,...,...,...
3409,数毛闲聊站,让车为人服务越来越好,0,2024/3/30 12:22,NaN,10
3410,我在梦里见过你821,红米还出汽车吗？,0,2024/3/30 12:22,1.0,8
3411,夏天的尾巴_Z,小米fold3为啥造的？,0,2024/3/30 12:22,NaN,12
3412,My咪西mi,suv,0,2024/3/30 12:22,NaN,3


In [4]:
# 删除 '内容' 列中包含空值的所有行
df = df.dropna()

# 删除重复
df = df.drop_duplicates()
df.shape

(1570, 6)

In [6]:
# 手动将点赞量离散化为分类变量
def classify_likes(likes):
    if likes <= 500:
        return 'Low'
    elif likes <= 1500:
        return 'Medium'
    else:
        return 'High'

df['LikeVolClass'] = df['likecount'].apply(classify_likes)

# 手动对标签进行数值化
df.loc[df['LikeVolClass'] == 'Low', 'LikeVolClass_Num'] = 1
df.loc[df['LikeVolClass'] == 'Medium', 'LikeVolClass_Num'] = 2
df.loc[df['LikeVolClass'] == 'High', 'LikeVolClass_Num'] = 3

# 将日期列转换为日期时间类型
df['date'] = pd.to_datetime(df['date'])

# 计算与参考日期（2024/3/30 12:22）的分钟差
reference_date = pd.Timestamp('2024-03-30 12:22')
df['TimeDifference(Minutes)'] = (df['date'] - reference_date) / pd.Timedelta(minutes=1) 
# 将时间差转换为分钟

df.head()


,username,content,likecount,date,amend_weight_sum,length,LikeVolClass,LikeVolClass_Num,TimeDifference(Minutes)
0,SupremeWow,小米su7强势破圈太火爆了，雷总得抓紧提高产能啊,3025,2024-03-30 12:22:00,2.0,24,High,3.0,0.0
1,及米侠,感觉这次黑稿不太多，看来车圈还是更干净一些！,1725,2024-03-30 12:23:00,1.0,22,High,3.0,1.0
3,米12U,雷总辛苦了 感谢您为大家做了一辆Dream car,660,2024-03-30 12:22:00,1.0,25,Medium,2.0,0.0
5,柴柴chaial啊,我今天看了一早上雷总的发布会，尤其在汽车发布会最后那个鞠躬，让人泪目,469,2024-03-30 12:25:00,1.0,34,Low,1.0,3.0
7,钟雨飞,年轻人特别喜欢SU7，定位太成功了！！！,297,2024-03-30 12:24:00,2.0,20,Low,1.0,2.0


In [8]:
 #特征和标签
X = df[['amend_weight_sum', 'length', 'TimeDifference(Minutes)']]
y = df['LikeVolClass_Num']


# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# 模型训练
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model_lr.fit(X_train, y_train) 


# 预测
y_pred = model_lr.predict(X_test)
# print(y_pred)


# 评估模型
accuracy_lr = accuracy_score(y_test, y_pred)
print(accuracy_lr)

1.0


/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
coefficients = model_lr.coef_
coefficients

array([[ 0.18284957, -0.02826754,  0.5092347 ],
       [-0.39038774,  0.06454653, -1.01841654],
       [ 0.20753818, -0.03627899,  0.50918185]])

随机森林

In [12]:

# 手动将点赞量离散化为分类变量
def classify_likes(likes):
    if likes <= 500:
        return 'Low'
    elif likes <= 1500:
        return 'Medium'
    else:
        return 'High'

df['LikeVolClass'] = df['likecount'].apply(classify_likes)

# # 手动对标签进行数值化
df.loc[df['LikeVolClass'] == 'Low', 'LikeVolClass_Num'] = 1
df.loc[df['LikeVolClass'] == 'Medium', 'LikeVolClass_Num'] = 2
df.loc[df['LikeVolClass'] == 'High', 'LikeVolClass_Num'] = 3

# 将日期列转换为日期时间类型
df['date'] = pd.to_datetime(df['date'])

# 计算与参考日期（2024/3/30 12:22）的分钟差
reference_date = pd.Timestamp('2024-03-30 12:22')
df['TimeDifference(Minutes)'] = (df['date'] - reference_date) / pd.Timedelta(minutes=1)  # 将时间差转换为分钟

 #特征和标签
X = df[['amend_weight_sum', 'length', 'TimeDifference(Minutes)']]
y = df['LikeVolClass_Num']
# y = df['LikeVolClass']

# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

# 模型训练
from sklearn.ensemble import RandomForestClassifier #引/入分类决策树模型
from sklearn.metrics import accuracy_score
model_rf = RandomForestClassifier(max_depth=3,n_estimators =10, min_samples_leaf=10,random_state=1)
model_rf.fit(X_train,y_train)


# 预测
y_pred = model_rf.predict(X_test)

# 评估模型
accuracy_rf = accuracy_score(y_test, y_pred)
accuracy_rf

1.0

In [13]:

features = X.columns
importances2 = model_rf.feature_importances_

importances_rf = pd.DataFrame()
importances_rf['Feature_Name'] = features
importances_rf['Feature_Importanc'] = importances2
importances_rf.sort_values('Feature_Importanc',ascending=False)
importances_rf

,Feature_Name,Feature_Importanc
0,amend_weight_sum,0.100379
1,length,0.381085
2,TimeDifference(Minutes),0.518536
